In [ ]:
# Install dependencies
!pip install torch
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

import os.path as osp
import torch
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.nn import SignedGCN
import time

# Set device to CPU
device = torch.device('cpu')

# Load BitcoinOTC dataset
name = 'BitcoinOTC-1'
path = osp.join('data', name)
dataset = BitcoinOTC(path, edge_window_size=1)

# Prepare positive and negative edges
pos_edge_indices, neg_edge_indices = [], []
for data in dataset:
    pos_edge_indices.append(data.edge_index[:, data.edge_attr > 0])
    neg_edge_indices.append(data.edge_index[:, data.edge_attr < 0])

# Move data to CPU
pos_edge_index = torch.cat(pos_edge_indices, dim=1).to(device)
neg_edge_index = torch.cat(neg_edge_indices, dim=1).to(device)

# Build and train SignedGCN model
model = SignedGCN(64, 64, num_layers=2, lamb=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_pos_edge_index, test_pos_edge_index = model.split_edges(pos_edge_index)
train_neg_edge_index, test_neg_edge_index = model.split_edges(neg_edge_index)
x = model.create_spectral_features(train_pos_edge_index, train_neg_edge_index).to(device)

def train():
    model.train()
    optimizer.zero_grad()
    z = model(x, train_pos_edge_index, train_neg_edge_index)
    loss = model.loss(z, train_pos_edge_index, train_neg_edge_index)
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        z = model(x, train_pos_edge_index, train_neg_edge_index)
    return model.test(z, test_pos_edge_index, test_neg_edge_index)

# Training loop with timing
start_time = time.time()
for epoch in range(101):
    loss = train()
    auc, f1 = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}')
cpu_time = time.time() - start_time
print(f"Training completed on CPU in {cpu_time:.2f} seconds")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.3/751.3 kB 39.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1E

Epoch: 000, Loss: 1.1718, AUC: 0.5000, F1: 0.0000
Epoch: 001, Loss: 1.1172, AUC: 0.4999, F1: 0.0025
Epoch: 002, Loss: 1.1096, AUC: 0.7199, F1: 0.7607
Epoch: 003, Loss: 1.1065, AUC: 0.5025, F1: 0.9467
Epoch: 004, Loss: 1.1036, AUC: 0.4998, F1: 0.9472
Epoch: 005, Loss: 1.1016, AUC: 0.5091, F1: 0.9470
Epoch: 006, Loss: 1.1000, AUC: 0.7433, F1: 0.9003
Epoch: 007, Loss: 1.0989, AUC: 0.6461, F1: 0.4918
Epoch: 008, Loss: 1.0979, AUC: 0.6422, F1: 0.4734
Epoch: 009, Loss: 1.0970, AUC: 0.7152, F1: 0.6637
Epoch: 010, Loss: 1.0960, AUC: 0.7790, F1: 0.8387
Epoch: 011, Loss: 1.0947, AUC: 0.7768, F1: 0.9196
Epoch: 012, Loss: 1.0937, AUC: 0.7525, F1: 0.9396
Epoch: 013, Loss: 1.0925, AUC: 0.7760, F1: 0.9229
Epoch: 014, Loss: 1.0911, AUC: 0.7900, F1: 0.8521
Epoch: 015, Loss: 1.0896, AUC: 0.7660, F1: 0.7730
Epoch: 016, Loss: 1.0880, AUC: 0.7867, F1: 0.8184
Epoch: 017, Loss: 1.0861, AUC: 0.8079, F1: 0.8890
Epoch: 018, Loss: 1.0840, AUC: 0.7924, F1: 0.9149
Epoch: 019, Loss: 1.0816, AUC: 0.8014, F1: 0.9084


In [ ]:
# Install dependencies
!pip install torch
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

import os.path as osp
import torch
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.nn import SignedGCN
import time

# Set device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load BitcoinOTC dataset
name = 'BitcoinOTC-1'
path = osp.join('data', name)
dataset = BitcoinOTC(path, edge_window_size=1)

# Prepare positive and negative edges
pos_edge_indices, neg_edge_indices = [], []
for data in dataset:
    pos_edge_indices.append(data.edge_index[:, data.edge_attr > 0])
    neg_edge_indices.append(data.edge_index[:, data.edge_attr < 0])

# Move data to GPU
pos_edge_index = torch.cat(pos_edge_indices, dim=1).to(device)
neg_edge_index = torch.cat(neg_edge_indices, dim=1).to(device)

# Build and train SignedGCN model
model = SignedGCN(64, 64, num_layers=2, lamb=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_pos_edge_index, test_pos_edge_index = model.split_edges(pos_edge_index)
train_neg_edge_index, test_neg_edge_index = model.split_edges(neg_edge_index)
x = model.create_spectral_features(train_pos_edge_index, train_neg_edge_index).to(device)

def train():
    model.train()
    optimizer.zero_grad()
    z = model(x, train_pos_edge_index, train_neg_edge_index)
    loss = model.loss(z, train_pos_edge_index, train_neg_edge_index)
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        z = model(x, train_pos_edge_index, train_neg_edge_index)
    return model.test(z, test_pos_edge_index, test_neg_edge_index)

# Training loop with timing
start_time = time.time()
for epoch in range(101):
    loss = train()
    auc, f1 = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}')
gpu_time = time.time() - start_time
print(f"Training completed on GPU in {gpu_time:.2f} seconds")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.3/751.3 kB 46.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1E

Epoch: 000, Loss: 1.1682, AUC: 0.5000, F1: 0.9473
Epoch: 001, Loss: 1.1162, AUC: 0.6834, F1: 0.7609
Epoch: 002, Loss: 1.1064, AUC: 0.5000, F1: 0.0000
Epoch: 003, Loss: 1.1040, AUC: 0.5000, F1: 0.0000
Epoch: 004, Loss: 1.1024, AUC: 0.5000, F1: 0.0000
Epoch: 005, Loss: 1.1006, AUC: 0.5000, F1: 0.0000
Epoch: 006, Loss: 1.0990, AUC: 0.5373, F1: 0.1994
Epoch: 007, Loss: 1.0979, AUC: 0.7296, F1: 0.8901
Epoch: 008, Loss: 1.0969, AUC: 0.7046, F1: 0.9471
Epoch: 009, Loss: 1.0960, AUC: 0.6959, F1: 0.9493
Epoch: 010, Loss: 1.0952, AUC: 0.7257, F1: 0.9401
Epoch: 011, Loss: 1.0946, AUC: 0.7376, F1: 0.9331
Epoch: 012, Loss: 1.0937, AUC: 0.7517, F1: 0.9291
Epoch: 013, Loss: 1.0927, AUC: 0.7550, F1: 0.9251
Epoch: 014, Loss: 1.0914, AUC: 0.7603, F1: 0.9103
Epoch: 015, Loss: 1.0900, AUC: 0.7733, F1: 0.8740
Epoch: 016, Loss: 1.0885, AUC: 0.7757, F1: 0.8461
Epoch: 017, Loss: 1.0870, AUC: 0.7829, F1: 0.8353
Epoch: 018, Loss: 1.0853, AUC: 0.7879, F1: 0.8609
Epoch: 019, Loss: 1.0832, AUC: 0.7912, F1: 0.8795


In [ ]:
import os.path as osp
import torch
import torch_xla.core.xla_model as xm
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.nn import SignedGCN
import time

device = xm.xla_device()

# Load dataset
name = 'BitcoinOTC-1'
path = osp.join('data', name)
dataset = BitcoinOTC(path, edge_window_size=1)

# Prepare edges
pos_edge_indices, neg_edge_indices = [], []
for data in dataset:
    pos_edge_indices.append(data.edge_index[:, data.edge_attr > 0])
    neg_edge_indices.append(data.edge_index[:, data.edge_attr < 0])

# Preload data onto TPU
pos_edge_index = torch.cat(pos_edge_indices, dim=1).to(device)
neg_edge_index = torch.cat(neg_edge_indices, dim=1).to(device)

# Model setup
model = SignedGCN(64, 64, num_layers=2, lamb=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
train_pos_edge_index, test_pos_edge_index = model.split_edges(pos_edge_index)
train_neg_edge_index, test_neg_edge_index = model.split_edges(neg_edge_index)
x = model.create_spectral_features(train_pos_edge_index, train_neg_edge_index).to(device)

def train():
    model.train()
    optimizer.zero_grad()
    z = model(x, train_pos_edge_index, train_neg_edge_index)
    loss = model.loss(z, train_pos_edge_index, train_neg_edge_index)
    loss.backward()
    xm.optimizer_step(optimizer)
    xm.mark_step()  # Signal TPU computation completion
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        z = model(x, train_pos_edge_index, train_neg_edge_index)
    return model.test(z, test_pos_edge_index, test_neg_edge_index)

# Training loop with timing
start_time = time.time()
for epoch in range(101):
    loss = train()
    auc, f1 = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}')
tpu_time = time.time() - start_time
print(f"Training completed on TPU in {tpu_time:.2f} seconds")


# Set device to CPU
device = torch.device('cpu')
# Set device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Preload data onto TPU
pos_edge_index = torch.cat(pos_edge_indices, dim=1).to(device)
neg_edge_index = torch.cat(neg_edge_indices, dim=1).to(device)


Epoch: 000, Loss: 1.1946, AUC: 0.5000, F1: 0.0000
Epoch: 001, Loss: 1.1260, AUC: 0.5023, F1: 0.0201
Epoch: 002, Loss: 1.1125, AUC: 0.5382, F1: 0.9354
Epoch: 003, Loss: 1.1082, AUC: 0.4999, F1: 0.9473
Epoch: 004, Loss: 1.1058, AUC: 0.4999, F1: 0.9473
Epoch: 005, Loss: 1.1039, AUC: 0.5382, F1: 0.9483
Epoch: 006, Loss: 1.1017, AUC: 0.6414, F1: 0.7568
Epoch: 007, Loss: 1.1001, AUC: 0.5733, F1: 0.4106
Epoch: 008, Loss: 1.0988, AUC: 0.6578, F1: 0.6562
Epoch: 009, Loss: 1.0979, AUC: 0.7281, F1: 0.8379
Epoch: 010, Loss: 1.0970, AUC: 0.7353, F1: 0.8747
Epoch: 011, Loss: 1.0962, AUC: 0.7378, F1: 0.8730
Epoch: 012, Loss: 1.0952, AUC: 0.7592, F1: 0.8803
Epoch: 013, Loss: 1.0944, AUC: 0.7663, F1: 0.8905
Epoch: 014, Loss: 1.0935, AUC: 0.7569, F1: 0.9056
Epoch: 015, Loss: 1.0925, AUC: 0.7560, F1: 0.9148
Epoch: 016, Loss: 1.0914, AUC: 0.7577, F1: 0.9102
Epoch: 017, Loss: 1.0902, AUC: 0.7779, F1: 0.8936
Epoch: 018, Loss: 1.0887, AUC: 0.7914, F1: 0.8638
Epoch: 019, Loss: 1.0870, AUC: 0.7950, F1: 0.8683
